In [2]:

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(
    "D:/Projects/Lattice interview 2/Summarise Report/",
    glob="*.txt",
    loader_cls=TextLoader
)
docs = loader.load()


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

db = FAISS.from_documents(chunks, embeddings)


C:\Users\amaan\AppData\Local\Temp\ipykernel_22660\730556291.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
d:\Projects\Lattice interview 2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
query = "Any day where work was not done on the project? If yes, then why?"
relevant_docs = db.similarity_search(query)

context = "\n\n".join([doc.page_content for doc in relevant_docs])
prompt = f"""You are going to Answer the question based on the context given below, be consize and try direct and correct answers only:\n\n{context}\n\nQuestion: {query}<|im_end|>
<|im_start|> assistant"""
print(context)


Daily Site Report - 2025-06-22

No work done, Bad weather holiday

Daily Site Report - 2025-06-22

The following issues were observed today:

1. Employee ID: 524562 raised a Safety Notice (Amber) regarding dim lights near the saw blade. This issue may potentially lead to accidents, and it is recommended that the lights be replaced or brightened for better visibility.

2. Employee ID: 524562 reported that additional manpower is required for removing windows. This task may be delayed if not addressed promptly.

Daily Site Report - 2025-06-19

Today's report highlights the following issues and tasks:

1. Safety Notice (Green) - Good Observation** - Employee ID 524572 reported good, clear exclusion zones and access throughout the site, contributing to a safe working environment.

2. Progress Photo** - Employee ID 524575 took progress photos of the ongoing RC walls construction, showing steady advancement in the structural phase.

3. Quality Defect (Material Handling)** - Employee ID 524578

In [10]:
import requests
LLAMA_URL = "http://127.0.0.1:8080/completion"
rompt = f"""
<|im_start|> You: Answer the question based on the context given below, be concise and try direct and correct answers only:\n\n{context}\n\nQuestion: {query}
Observations:
<|im_end|>
<|im_start|> assistant
"""
payload = {
                "prompt": prompt,
                "temperature": 0.7, 
                "n_predict": 512,  # for longer reports
                "stream": False,
                "stop": ["<|im_end|>"],
            }
response = requests.post(LLAMA_URL, json=payload, timeout=120) # Increased timeout
response.raise_for_status()
result = response.json()
ai_generated_report = result.get("content", "").strip()
print(ai_generated_report)

: Yes, according to the Daily Site Report for 2025-06-22, no work was done due to bad weather.
